In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

# RNN(Recurrent Neural Network)
`h_now = f(h_prev, x_now) = tanh(h_prev * w_h + x_now * w_x + b)`
`y_now = h_now * w_y + b_y`

- one-to-many: RNN 층에는 첫 시점에만 값이 들어오며, 이 값으로 출력 시퀀스를 만든다.
    - 이미지를 받아 이를 설명하는 단어 시퀀스를 출력.
- many-to-many: RNN 층에 여러 시점에 값이 들어오며, 이 값으로 출력 시퀀스를 만든다.
    - 토큰 시퀀스를 받아, 각 토큰의 개체명을 출력.
- many-to-one: RNN 층에 여러 시점에 값이 들어오며, 이 값으로 최종 시점에 출력을 만든다.
    - 일정 기간 월별 판매량 데이터로 다음 달 판매량 예측.

이번 노트북에서는 `x=[-10, 10]` 범위 sin 함수로 학습한 RNN 모델에게, 다른 범위의 x를 주고
값을 예측하도록 시킨 다음 이를 그 범위의 cos 함수와 비교한다.

## 데이터 준비

In [ ]:
def split_seqeunce(sequence, step):
    """데이터셋용 시퀀스 분할.

    x, y = sequence[i:i + step], sequence[i + step]
    시퀀스의 일정 부분을 보고 그 다음엔 뭐가 올지 추측하는 모델을 위한 데이터셋 생성 헬퍼 함수.

    :param sequence: 분할 대상.
    :param step: 분할 크기.
    :return: 분할된 부분과 그 다음 원소의 슬라이스들.
    """
    x, y = [], []

    for i in range(len(sequence)):
        end_idx = i + step
        if end_idx > len(sequence) - 1:
            break
        x.append(sequence[i:end_idx])
        y.append(sequence[end_idx])

    return np.array(x), np.array(y)


def preprocess(dataset: np.ndarray, new_dim_size: int) -> np.ndarray:
    """RNN 레이어 입력 요구 사항을 맞추기 위한 전처리.

    :param dataset: ndim=2의 ndarray
    :param new_dim_size: 추가된 차원의 크기.
    :return: ndim = (ndim + 1)로 형태가 바뀐 ndarray
    """
    return dataset.reshape(*dataset.shape, new_dim_size)

In [ ]:
x_ticks = 15  # RNN이 1회 추론에 입력 받게 될 데이터.
n_features = 1  # 전처리 단계와 출력층의 크기로서도 써먹는다.

In [ ]:
x = np.arange(start=-10, stop=10, step=0.1)
dataset_y = np.sin(x)
dataset_x, dataset_y = split_seqeunce(dataset_y, x_ticks)

In [ ]:
print(dataset_x.shape)  # shape: (len(x) / x_ticks, x_ticks)
print(dataset_y.shape)

In [ ]:
# RNN 요구 shape(ndim=3)을 맞추기 위해 차원 추가.
dataset_x = preprocess(dataset_x, n_features)

In [ ]:
input_layer = keras.layers.Input(shape=dataset_x.shape[1:])
rnn_layer = keras.layers.SimpleRNN(
    units=10,
    return_sequences=False,
    input_shape=(x_ticks, n_features))(input_layer)
dense_layer = keras.layers.Dense(n_features)(rnn_layer)
rnn_model = keras.Model(inputs=input_layer, outputs=dense_layer)
rnn_model.compile(optimizer=keras.optimizers.legacy.Adam(), loss="mse")

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="loss",
    patience=5,
    mode="auto")
history = rnn_model.fit(dataset_x, dataset_y, epochs=1000, callbacks=[early_stopping])

## 훈련 결과 확인

In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.legend(loc="upper right")
plt.show()

## 테스트셋 검증

In [ ]:
x = np.arange(10, 20, 0.1)
y = np.cos(x)

In [ ]:
test_x, test_y = split_seqeunce(y, x_ticks)
print(test_x.shape, test_y.shape)
# Preprocess
test_x = preprocess(test_x, n_features)

In [ ]:
pred_y = rnn_model.predict(test_x)

In [ ]:
# x[:x_ticks] 분만큼은 범위 문제로 테스트셋에선 예측할 수 없으니, 정답과 예측 그래프를 맞추기 위해
# 이 부분은 앞에 정답을 끼워넣는다.
len_fixed_pred_y = np.append(y[:x_ticks], pred_y)

In [ ]:
plt.plot(x, y, label="groundtruth", color="blue")
plt.plot(x, len_fixed_pred_y, label="predictions", color="orange")
plt.legend(loc="upper left")
plt.ylim(-2, 2)
plt.show()

# LSTM 모델 생성 및 예측

RNN 모델의 한계로는 시퀀스가 길어질수록 최근의 데이터 영향이 강하고, 시퀀스 앞쪽의 데이터의 영향은 약해지는
장기의존성(Long term dependency)이 생긴다는 점이다. 이를 해소하기 위한 모델 구조로 LSTM이 제안됐다.

Long-Short Term Model이란 이름에서 알 수 있듯, 단기 상태이던 은닉 상태(hidden state)에
장기 상태인 셀 상태(cell state)를 추가하여 RNN 모델의 단점을 해소했다. 이런 셀 상태를 계산하기 위해
'입력, 삭제, 출력 게이트'라는 3개의 게이트 모델에 도입했다.

LSTM에서 사용되는 활성화 함수는 두 가지다: sigmoid, tanh. 각각의 함수의 범위는 sig가 `[0, 1]`,
tanh가 `[-1, 1]`이다. 여기서 sig는 정보를 얼마나 약화시킬지 감쇠도를, tanh는 정보를 단기 상태 범위로
스케일을 축소시킨다.

  - 입력 게이트(input gate): 장기 상태에 추가할 정보량(수명)을 의미.
    - i_cur = sig(x_cur, h_prev)
    - g_cur = tanh(x_cur, h_prev)
  - 삭제 게이트(forget gate): 장기 상태를 얼마나 잊을지 감쇠도.
    - f_cur = sig(x_cur, h_prev)
  - 출력 게이트(output gate): 단기 상태를 얼마나 잊을지 감쇠도.
    - o_cur = sig(x_cur, h_prev)

  - 단기 상태: 이전 시점까지 끌고 온 장기적 정보(c_cur)를 단기적 정보 범위로 바꾼다.
    - h_cur = tanh(c_cur) * o_cur
  - 장기 상태: 이전 정보에 감쇠도를 적용 후, 이번 시점 정보를 추가한다.
    - c_cur = c_prev * f_cur + i_cur * g_cur


모델 구조만 RNN 레이어를 LSTM 레이어로 변경하면 된다.

In [ ]:
lstm_layer = keras.layers.LSTM(
    units=10,
    return_sequences=False,
    input_shape=(x_ticks, n_features))(input_layer)
dense_layer = keras.layers.Dense(n_features)(lstm_layer)
lstm_model = keras.Model(inputs=input_layer, outputs=dense_layer)
lstm_model.compile(optimizer=keras.optimizers.legacy.Adam(), loss="mse")

In [ ]:
history = lstm_model.fit(dataset_x, dataset_y, epochs=1000, callbacks=[early_stopping])

In [ ]:
pred_y = lstm_model.predict(test_x)

In [ ]:
len_fixed_pred_y = np.append(y[:x_ticks], pred_y)
plt.plot(x, y, label="groundtruth", color="blue")
plt.plot(x, len_fixed_pred_y, label="predictions", color="orange")
plt.legend(loc="upper left")
plt.ylim(-2, 2)
plt.show()